In [7]:
import sys
sys.path.append('/home/ian/code/github')

## 获取症状列表

In [8]:
# from utils.pymongo_wrapper import pymongo_wrapper as pmw

# client = pmw()

# zztable = client.get_collection('jiankang39', 'zz')

# zzlist = [i['症状名称'] for i in client.find_all(zztable, fieldlist=['症状名称'])]



# from utils.pickle_wrapper import pickle_wrapper as picklew

# picklew.dump2file(zzlist,'症状列表.pkl')
from utils.pickle_wrapper import pickle_wrapper as picklew
zzlist = picklew.loadfromfile('症状列表.pkl')
zzlist[:5]

['脑血管痉挛', '脑干出血', '多梦', '失眠', '脑缺氧']

### 通过编辑距离计算相似度

In [48]:
from utils.nlp.nlp_wrapper import nlp_wrapper as nlpw
from utils.pandas_wrapper import pandas_wrapper as pdw
def get_zzs_by_editdis(s1,zzlist, n=3):
    e = list(map(nlpw.cal_editdistance, [s1]*len(zzlist), zzlist))
    df = pdw.build_df_with_dict({'zz':zzlist,'e':e})
    return list(pdw.sort_by_column(df,'e').loc[:,'zz'][:n].values)

In [24]:
get_zzs_by_editdis('睡不着觉')

['气不摄血', '饮不解渴', '不举']

In [25]:
get_zzs_by_editdis('我经常失眠')

['失眠', '色素脱失', '意识丧失']

### 通过word2vec模糊推荐症状

In [29]:
import gensim
import jieba
import numpy as np
from scipy.linalg import norm
 
model_file = '/home/ian/code/github/data/word2vec/news_12g_baidubaike_20g_novel_90g_embedding_64.bin'
model = gensim.models.KeyedVectors.load_word2vec_format(model_file, binary=True)


In [36]:
 
def vector_similarity(s1, s2):
    def sentence_vector(s):
        words = jieba.lcut(s)
        v = np.zeros(64)
        for word in words:
            if word in model:
                v += model[word]
        v /= len(words)
        return v
    
    v1, v2 = sentence_vector(s1), sentence_vector(s2)
    return np.dot(v1, v2) / (norm(v1) * norm(v2))


In [51]:
get_zzs_by_word2vec('睡不着觉')

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


['失眠', '失眠多梦']

In [52]:
get_zzs_by_word2vec('我经常失眠')

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


['经常流鼻血', '反复发作的带状疱疹']

### 得到模糊症状匹配
如何提高准确率？  
* 在模型使用过程中，排除影响模型的个别word  
* zzlist  
保证zzlist都是可问的，就是说根据用户的特征（孕妇，老人，男女）有针对的zzlist

In [50]:
def get_zzs_by_word2vec(s1,zzlist,n=2):
    e = list(map(vector_similarity, [s1]*len(zzlist), zzlist))
    df = pdw.build_df_with_dict({'zz':zzlist,'e':e})
    return list(pdw.sort_by_column(df,'e',ascending=False).loc[:,'zz'][:n].values)

In [60]:
import re
def get_zzs(s1,zzlist):
    s1 = re.sub(r'我','',s1)
    return list(set(get_zzs_by_word2vec(s1,zzlist)+get_zzs_by_editdis(s1,zzlist)))

In [61]:
get_zzs('我睡不着觉')

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


['气不摄血', '失眠多梦', '饮不解渴', '不举', '失眠']

In [62]:
get_zzs('我脸有点肿')

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


['喉头水肿', '脸冻伤', '面部水肿', '痛性脸抽搐', '脸部疼痛']

In [63]:
get_zzs('我有点发烧')

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


['四肢发冷', '喉咙痛', '手心发热', '小腿发胀', '流鼻血']

In [46]:
e = list(map(vector_similarity, ['我牙疼']*len(zzlist), zzlist))
e[:5]

/home/ian/installed/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


[0.25886484774584,
 0.20469593383021958,
 0.09579593970598164,
 0.2230319561722977,
 0.09064157739034769]

In [47]:
df = pdw.build_df_with_dict({'zz':zzlist,'e':e})
list(pdw.sort_by_column(df,'e',ascending=False).loc[:,'zz'][:5].values)

['双眼难以忍受的奇痒', '上呼吸道卡他症状', '日轻夜重的瘙痒', '反复发作的带状疱疹', '足的放射性疼痛']

In [40]:
pdw.sort_by_column(df,'e')

,zz,e
541,眼底发现黄斑呈带灰色变色,-0.268249
2805,足跟蓝黑色或黑色斑点,-0.261248
2266,诊断室中所测血压始终增高,-0.258950
776,鼻唇沟变浅,-0.245501
754,鼻中隔向一侧或两则弯曲,-0.244906
1889,蓝色、蓝灰色或蓝黑色丘疹,-0.242348
1722,皮肤水疱，后形成黑色干痂,-0.241613
2934,骶部可有皮赘，形成尾巴,-0.239277
1044,面容呈灰色,-0.239224
1533,成人大便绿色,-0.237868
